<a href="https://colab.research.google.com/github/PadmarajBhat/Rapids.AI/blob/master/notMNIST.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

From : https://www.ritchieng.com/machine-learning/deep-learning/tensorflow/notmnist/

import the downloading the notMNIST modules

In [0]:
# These are all the modules we'll be using later. Make sure you can import them
# before proceeding further.
from __future__ import print_function
import matplotlib.pyplot as plt
import numpy as np
import os
import sys
import tarfile
from IPython.display import display, Image
from scipy import ndimage
from sklearn.linear_model import LogisticRegression
from six.moves.urllib.request import urlretrieve
from six.moves import cPickle as pickle

# Config the matlotlib backend as plotting inline in IPython
%matplotlib inline

url = 'http://commondatastorage.googleapis.com/books1000/'
last_percent_reported = None

def download_progress_hook(count, blockSize, totalSize):
  """A hook to report the progress of a download. This is mostly intended for users with
  slow internet connections. Reports every 1% change in download progress.
  """
  global last_percent_reported
  percent = int(count * blockSize * 100 / totalSize)

  if last_percent_reported != percent:
    if percent % 5 == 0:
      sys.stdout.write("%s%%" % percent)
      sys.stdout.flush()
    else:
      sys.stdout.write(".")
      sys.stdout.flush()
      
    last_percent_reported = percent
        
def maybe_download(filename, expected_bytes, force=False):
  """Download a file if not present, and make sure it's the right size."""
  if force or not os.path.exists(filename):
    print('Attempting to download:', filename) 
    filename, _ = urlretrieve(url + filename, filename, reporthook=download_progress_hook)
    print('\nDownload Complete!')
  statinfo = os.stat(filename)
  if statinfo.st_size == expected_bytes:
    print('Found and verified', filename)
  else:
    raise Exception(
      'Failed to verify ' + filename + '. Can you get to it with a browser?')
  return filename

train_filename = maybe_download('notMNIST_large.tar.gz', 247336696)
test_filename = maybe_download('notMNIST_small.tar.gz', 8458043)

Attempting to download: notMNIST_large.tar.gz
0%....5%....10%....15%....20%....25%....30%....35%....40%....45%....50%....55%....60%....65%....70%....75%....80%....85%....90%....95%....100%
Download Complete!
Found and verified notMNIST_large.tar.gz
Attempting to download: notMNIST_small.tar.gz
0%....5%....10%....15%....20%....25%....30%....35%....40%....45%....50%....55%....60%....65%....70%....75%....80%....85%....90%....95%....100%
Download Complete!
Found and verified notMNIST_small.tar.gz


### Extract them

In [0]:
num_classes = 10
np.random.seed(133)

def maybe_extract(filename, force=False):
  root = os.path.splitext(os.path.splitext(filename)[0])[0]  # remove .tar.gz
  if os.path.isdir(root) and not force:
    # You may override by setting force=True.
    print('%s already present - Skipping extraction of %s.' % (root, filename))
  else:
    print('Extracting data for %s. This may take a while. Please wait.' % root)
    tar = tarfile.open(filename)
    sys.stdout.flush()
    tar.extractall()
    tar.close()
  data_folders = [
    os.path.join(root, d) for d in sorted(os.listdir(root))
    if os.path.isdir(os.path.join(root, d))]
  if len(data_folders) != num_classes:
    raise Exception(
      'Expected %d folders, one per class. Found %d instead.' % (
        num_classes, len(data_folders)))
  print(data_folders)
  return data_folders
  
train_folders = maybe_extract(train_filename)
test_folders = maybe_extract(test_filename)

Extracting data for notMNIST_large. This may take a while. Please wait.
['notMNIST_large/A', 'notMNIST_large/B', 'notMNIST_large/C', 'notMNIST_large/D', 'notMNIST_large/E', 'notMNIST_large/F', 'notMNIST_large/G', 'notMNIST_large/H', 'notMNIST_large/I', 'notMNIST_large/J']
Extracting data for notMNIST_small. This may take a while. Please wait.
['notMNIST_small/A', 'notMNIST_small/B', 'notMNIST_small/C', 'notMNIST_small/D', 'notMNIST_small/E', 'notMNIST_small/F', 'notMNIST_small/G', 'notMNIST_small/H', 'notMNIST_small/I', 'notMNIST_small/J']


In [0]:
Image(filename="notMNIST_small/A/Q0NXaWxkV29yZHMtQm9sZEl0YWxpYy50dGY=.png")

In [0]:
os.listdir("notMNIST_small/")

['E', 'D', 'C', 'G', 'I', 'A', 'H', 'F', 'J', 'B']

How to display multiple Images in a cell?
https://stackoverflow.com/a/49487396/8693106

In [0]:
import random
listOfFiles = []
for (dirpath, dirnames, filenames) in os.walk("notMNIST_small/"):
  listOfFiles += [os.path.join(dirpath, file) for file in filenames]
  
  for f in listOfFiles:
    if random.randint(0,1000) == random.randint(0,1000):
      print(f)
      display(Image(filename=f))
      #break
  #break

notMNIST_small/E/Q2xpY2hlZUNFLU9ibGlxdWUub3Rm.png


notMNIST_small/E/Q2FycGFsIFR1bm5lbC50dGY=.png


notMNIST_small/E/Q2FwaXRhbHMudHRm.png


notMNIST_small/E/RXJhc0JRLUJvb2sub3Rm.png


notMNIST_small/D/RGVtb2NyYXRpY2FCb2xkT2xkc3R5bGUub3Rm.png


...

notMNIST_small/F/RmluZUhhbmRQbGFpbi5vdGY=.png


notMNIST_small/F/QXRsYXMgb2YgdGhlIE1hZ2kudHRm.png


notMNIST_small/J/RnJhbmtUaW1lcy1JdGFsaWMudHRm.png


notMNIST_small/B/R291ZHkgSGFuZHRvb2xlZC5wZmI=.png


notMNIST_small/B/RGF5dHJpcHBlciBQbGFpbi50dGY=.png


In [0]:
import imageio

image_size = 28  # Pixel width and height.
pixel_depth = 255.0  # Number of levels per pixel.

def load_letter(folder, min_num_images):
  """Load the data for a single letter label."""
  image_files = os.listdir(folder)
  dataset = np.ndarray(shape=(len(image_files), image_size, image_size),
                         dtype=np.float32)
  print(folder)
  num_images = 0
  for image in image_files:
    image_file = os.path.join(folder, image)
    try:
      image_data = (imageio.imread(image_file).astype(float) - 
                    pixel_depth / 2) / pixel_depth
      if image_data.shape != (image_size, image_size):
        raise Exception('Unexpected image shape: %s' % str(image_data.shape))
      dataset[num_images, :, :] = image_data
      num_images = num_images + 1
    except (IOError, ValueError) as e:
      print('Could not read:', image_file, ':', e, '- it\'s ok, skipping.')
    
  dataset = dataset[0:num_images, :, :]
  if num_images < min_num_images:
    raise Exception('Many fewer images than expected: %d < %d' %
                    (num_images, min_num_images))
    
  print('Full dataset tensor:', dataset.shape)
  print('Mean:', np.mean(dataset))
  print('Standard deviation:', np.std(dataset))
  return dataset
        
def maybe_pickle(data_folders, min_num_images_per_class, force=False):
  dataset_names = []
  for folder in data_folders:
    set_filename = folder + '.pickle'
    dataset_names.append(set_filename)
    if os.path.exists(set_filename) and not force:
      # You may override by setting force=True.
      print('%s already present - Skipping pickling.' % set_filename)
    else:
      print('Pickling %s.' % set_filename)
      dataset = load_letter(folder, min_num_images_per_class)
      try:
        with open(set_filename, 'wb') as f:
          pickle.dump(dataset, f, pickle.HIGHEST_PROTOCOL)
      except Exception as e:
        print('Unable to save data to', set_filename, ':', e)
  
  return dataset_names

train_datasets = maybe_pickle(train_folders, 45000)
test_datasets = maybe_pickle(test_folders, 1800)

Pickling notMNIST_large/A.pickle.
notMNIST_large/A
Could not read: notMNIST_large/A/SG90IE11c3RhcmQgQlROIFBvc3Rlci50dGY=.png : Could not find a format to read the specified file in mode 'i' - it's ok, skipping.
Could not read: notMNIST_large/A/RnJlaWdodERpc3BCb29rSXRhbGljLnR0Zg==.png : Could not find a format to read the specified file in mode 'i' - it's ok, skipping.
Could not read: notMNIST_large/A/Um9tYW5hIEJvbGQucGZi.png : Could not find a format to read the specified file in mode 'i' - it's ok, skipping.
Full dataset tensor: (52909, 28, 28)
Mean: -0.12825015
Standard deviation: 0.4431204
Pickling notMNIST_large/B.pickle.
notMNIST_large/B
Could not read: notMNIST_large/B/TmlraXNFRi1TZW1pQm9sZEl0YWxpYy5vdGY=.png : Could not find a format to read the specified file in mode 'i' - it's ok, skipping.
Full dataset tensor: (52911, 28, 28)
Mean: -0.0075630345
Standard deviation: 0.45449153
Pickling notMNIST_large/C.pickle.
notMNIST_large/C
Full dataset tensor: (52912, 28, 28)
Mean: -0.1422

In [0]:
# intall miniconda
!wget -c https://repo.continuum.io/miniconda/Miniconda3-4.5.4-Linux-x86_64.sh
!chmod +x Miniconda3-4.5.4-Linux-x86_64.sh
!bash ./Miniconda3-4.5.4-Linux-x86_64.sh -b -f -p /usr/local

'''
# install RAPIDS packages
!conda install -q -y --prefix /usr/local -c conda-forge \
  -c rapidsai-nightly/label/cuda10.0 -c nvidia/label/cuda10.0 \
  cudf cuml
'''

!conda install -c nvidia/label/cuda10.0 -c rapidsai/label/cuda10.0 \
  -c numba -c conda-forge -c defaults cudf

--2019-07-24 23:59:35--  https://repo.continuum.io/miniconda/Miniconda3-4.5.4-Linux-x86_64.sh
Resolving repo.continuum.io (repo.continuum.io)... 104.18.200.79, 104.18.201.79, 2606:4700::6812:c84f, ...
Connecting to repo.continuum.io (repo.continuum.io)|104.18.200.79|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 58468498 (56M) [application/x-sh]
Saving to: ‘Miniconda3-4.5.4-Linux-x86_64.sh’

Miniconda3-4.5.4-Li 100%[===================>]  55.76M   128MB/s    in 0.4s    

2019-07-24 23:59:36 (128 MB/s) - ‘Miniconda3-4.5.4-Linux-x86_64.sh’ saved [58468498/58468498]

PREFIX=/usr/local
installing: python-3.6.5-hc3d631a_2 ...
Python 3.6.5 :: Anaconda, Inc.
installing: ca-certificates-2018.03.07-0 ...
installing: conda-env-2.6.0-h36134e3_1 ...
installing: libgcc-ng-7.2.0-hdf63c60_3 ...
installing: libstdcxx-ng-7.2.0-hdf63c60_3 ...
installing: libffi-3.2.1-hd88cf55_4 ...
installing: ncurses-6.1-hf484d3e_0 ...
installing: openssl-1.0.2o-h20670df_0 ...
installing: tk-

In [0]:
# set environment vars
import sys, os, shutil
sys.path.append('/usr/local/lib/python3.6/site-packages/')
os.environ['NUMBAPRO_NVVM'] = '/usr/local/cuda/nvvm/lib64/libnvvm.so'
os.environ['NUMBAPRO_LIBDEVICE'] = '/usr/local/cuda/nvvm/libdevice/'

# copy .so files to current working dir
for fn in ['libcudf.so', 'librmm.so']:
  shutil.copy('/usr/local/lib/'+fn, os.getcwd())

In [0]:
!conda install -c numba -c conda-forge -c nvidia -c rapidsai -c defaults cudf

Traceback (most recent call last):
  File "/usr/local/bin/conda", line 7, in <module>
    from conda.cli import main
ModuleNotFoundError: No module named 'conda'


In [0]:
# intall miniconda
'''!wget -c https://repo.continuum.io/miniconda/Miniconda3-4.5.4-Linux-x86_64.sh
!chmod +x Miniconda3-4.5.4-Linux-x86_64.sh
!bash ./Miniconda3-4.5.4-Linux-x86_64.sh -b -f -p /usr/local
'''
!wget  https://github.com/rapidsai/notebooks-extended/raw/master/utils/rapids-colab.sh
!chmod +x rapids-colab.sh
!bash ./rapids-colab.sh -b -f -p /usr/local

--2019-07-24 00:33:35--  https://github.com/rapidsai/notebooks-extended/raw/master/utils/rapids-colab.sh
Resolving github.com (github.com)... 140.82.114.3
Connecting to github.com (github.com)|140.82.114.3|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/rapidsai/notebooks-extended/master/utils/rapids-colab.sh [following]
--2019-07-24 00:33:35--  https://raw.githubusercontent.com/rapidsai/notebooks-extended/master/utils/rapids-colab.sh
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1548 (1.5K) [text/plain]
Saving to: ‘rapids-colab.sh.1’

rapids-colab.sh.1   100%[===================>]   1.51K  --.-KB/s    in 0s      

2019-07-24 00:33:35 (285 MB/s) - ‘rapids-colab.sh.1’ saved [1548/1548]

File ‘env-c

In [0]:
!pip uninstall -y xgboost dask distributed
!# intall miniconda
!echo "Installing conda"
!wget https://repo.continuum.io/miniconda/Miniconda3-4.5.4-Linux-x86_64.sh
!chmod +x Miniconda3-4.5.4-Linux-x86_64.sh
!bash ./Miniconda3-4.5.4-Linux-x86_64.sh -b -f -p /usr/local
!
!echo "Installing RAPIDS packages"
!echo "Please standby, this will take a few minutes..."
!# install RAPIDS packages
!conda install -y --prefix /usr/local \
  -c rapidsai-nightly/label/xgboost -c rapidsai-nightly -c nvidia -c conda-forge \
  python=3.6 cudatoolkit=10.0 \
  cudf cuml cugraph gcsfs pynvml \
  dask-cudf dask-cuml \
  rapidsai/label/xgboost::xgboost=>0.9
  
!echo "Copying shared object files to /usr/lib"
!# copy .so files to /usr/lib, where Colab's Python looks for libs
!cp /usr/local/lib/libcudf.so /usr/lib/libcudf.so
!cp /usr/local/lib/librmm.so /usr/lib/librmm.so
!cp /usr/local/lib/libxgboost.so /usr/lib/libxgboost.so
!cp /usr/local/lib/libnccl.so /usr/lib/libnccl.so

Skipping xgboost as it is not installed.
Skipping dask as it is not installed.
Skipping distributed as it is not installed.
Installing conda
--2019-07-24 00:41:08--  https://repo.continuum.io/miniconda/Miniconda3-4.5.4-Linux-x86_64.sh
Resolving repo.continuum.io (repo.continuum.io)... 104.18.201.79, 104.18.200.79, 2606:4700::6812:c84f, ...
Connecting to repo.continuum.io (repo.continuum.io)|104.18.201.79|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 58468498 (56M) [application/x-sh]
Saving to: ‘Miniconda3-4.5.4-Linux-x86_64.sh.2’

Miniconda3-4.5.4-Li 100%[===================>]  55.76M   120MB/s    in 0.5s    

2019-07-24 00:41:09 (120 MB/s) - ‘Miniconda3-4.5.4-Linux-x86_64.sh.2’ saved [58468498/58468498]

PREFIX=/usr/local
installing: python-3.6.5-hc3d631a_2 ...
Python 3.6.5 :: Anaconda, Inc.
installing: ca-certificates-2018.03.07-0 ...
installing: conda-env-2.6.0-h36134e3_1 ...
installing: libgcc-ng-7.2.0-hdf63c60_3 ...
installing: libstdcxx-ng-7.2.0-hdf63c

In [0]:
!$PATH
!$PYTHONPATH

/bin/bash: /usr/local/nvidia/bin:/usr/local/cuda/bin:/usr/local/sbin:/usr/local/bin:/usr/sbin:/usr/bin:/sbin:/bin:/tools/node/bin:/tools/google-cloud-sdk/bin:/opt/bin: No such file or directory
/bin/bash: /env/python: No such file or directory


In [0]:
!ls /usr/local/bin/py*

/usr/local/bin/pyami_sendmail	      /usr/local/bin/pyrsa-sign
/usr/local/bin/pybabel		      /usr/local/bin/pyrsa-verify
/usr/local/bin/pycc		      /usr/local/bin/pytest
/usr/local/bin/pydoc		      /usr/local/bin/py.test
/usr/local/bin/pydoc3		      /usr/local/bin/python
/usr/local/bin/pydoc3.6		      /usr/local/bin/python3
/usr/local/bin/pygmentize	      /usr/local/bin/python3.6
/usr/local/bin/pyjwt		      /usr/local/bin/python3.6-config
/usr/local/bin/pyrsa-decrypt	      /usr/local/bin/python3.6m
/usr/local/bin/pyrsa-decrypt-bigfile  /usr/local/bin/python3.6m-config
/usr/local/bin/pyrsa-encrypt	      /usr/local/bin/python3-config
/usr/local/bin/pyrsa-encrypt-bigfile  /usr/local/bin/pyvenv
/usr/local/bin/pyrsa-keygen	      /usr/local/bin/pyvenv-3.6
/usr/local/bin/pyrsa-priv2pub


In [0]:
!python ; import cudf

Python 3.6.7 | packaged by conda-forge | (default, Jul  2 2019, 02:18:42) 
[GCC 7.3.0] on linux
Type "help", "copyright", "credits" or "license" for more information.
>>> import cudf
Traceback (most recent call last):
  File "<stdin>", line 1, in <module>
  File "/usr/local/lib/python3.6/site-packages/cudf/__init__.py", line 5, in <module>
    from cudf import dataframe, datasets
  File "/usr/local/lib/python3.6/site-packages/cudf/dataframe/__init__.py", line 3, in <module>
    from cudf.dataframe import (
  File "/usr/local/lib/python3.6/site-packages/cudf/dataframe/buffer.py", line 6, in <module>
    from cudf.utils import cudautils, utils
  File "/usr/local/lib/python3.6/site-packages/cudf/utils/cudautils.py", line 11, in <module>
    from cudf.utils.utils import (
  File "/usr/local/lib/python3.6/site-packages/cudf/utils/utils.py", line 6, in <module>
    import pandas as pd
  File "/usr/local/lib/python3.6/site-packages/pandas/__init__.py", line 17, in <module>
    "Unable to impo

In [0]:
import cudf
df = dask-cudf.DataFrame()
df['key'] = [0, 1, 2, 3, 4]
df['val'] = [float(i + 10) for i in range(5)]  # insert column
print(df)

ModuleNotFoundError: ignored

In [1]:
!wget https://repo.anaconda.com/archive/Anaconda3-2019.07-Linux-x86_64.sh

--2019-07-25 01:17:54--  https://repo.anaconda.com/archive/Anaconda3-2019.07-Linux-x86_64.sh
Resolving repo.anaconda.com (repo.anaconda.com)... 104.16.130.3, 104.16.131.3, 2606:4700::6810:8303, ...
Connecting to repo.anaconda.com (repo.anaconda.com)|104.16.130.3|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 541906131 (517M) [application/x-sh]
Saving to: ‘Anaconda3-2019.07-Linux-x86_64.sh’

Anaconda3-2019.07-L 100%[===================>] 516.80M   134MB/s    in 3.9s    

2019-07-25 01:17:58 (133 MB/s) - ‘Anaconda3-2019.07-Linux-x86_64.sh’ saved [541906131/541906131]



In [0]:
!chmod +x Anaconda3-2019.07-Linux-x86_64.sh

In [3]:
!./Anaconda3-2019.07-Linux-x86_64.sh 



Welcome to Anaconda3 2019.07

In order to continue the installation process, please review the license
agreement.
Please, press ENTER to continue
>>> 
Anaconda End User License Agreement

Copyright 2015, Anaconda, Inc.

All rights reserved under the 3-clause BSD License:

Redistribution and use in source and binary forms, with or without modification,
 are permitted provided that the following conditions are met:

  * Redistributions of source code must retain the above copyright notice, this 
list of conditions and the following disclaimer.
  * Redistributions in binary form must reproduce the above copyright notice, th
is list of conditions and the following disclaimer in the documentation and/or o
ther materials provided with the distribution.
  * Neither the name of Anaconda, Inc. ("Anaconda, Inc.") nor the names of its c
ontributors may be used to endorse or promote products derived from this softwar
e without specific prior written permission.

THIS SOFTWARE IS PROVIDED BY THE C

In [4]:
!echo $PATH

/usr/local/nvidia/bin:/usr/local/cuda/bin:/usr/local/sbin:/usr/local/bin:/usr/sbin:/usr/bin:/sbin:/bin:/tools/node/bin:/tools/google-cloud-sdk/bin:/opt/bin


In [5]:
import os
os.environ['PATH'] = ":/root/anaconda3/bin:" +os.environ['PATH']
!echo $PATH

:/root/anaconda3/bin:/usr/local/nvidia/bin:/usr/local/cuda/bin:/usr/local/sbin:/usr/local/bin:/usr/sbin:/usr/bin:/sbin:/bin:/tools/node/bin:/tools/google-cloud-sdk/bin:/opt/bin


In [6]:
!conda install -y --prefix /root/anaconda3 \
  -c rapidsai-nightly/label/xgboost -c rapidsai-nightly -c nvidia -c conda-forge \
  python=3.6 cudatoolkit=10.0 \
  cudf cuml cugraph gcsfs pynvml \
  dask-cudf dask-cuml \
  rapidsai/label/xgboost::xgboost=>0.9


UnsatisfiableError: The following specifications were found to be incompatible with a past
explicit spec that is not an explicit spec in this operation (python):

  - cudf -> pyarrow=0.12.1 -> parquet-cpp=1.5.1 -> arrow-cpp[version='>=0.11.0,>=0.11.1,<0.12.0a0,>=0.12.0,>=0.12.0,<0.13.0a0'] -> numpy[version='>=1.11.3,<2.0a0,>=1.14,<2.0a0'] -> python[version='>=3.6,<3.7.0a0,>=3.7,<3.8.0a0']
  - cugraph -> cudf[version='0.7.*,0.8.*,0.9.*'] -> pyarrow=0.12.1 -> parquet-cpp=1.5.1 -> arrow-cpp[version='>=0.11.0,>=0.11.1,<0.12.0a0,>=0.12.0,>=0.12.0,<0.13.0a0'] -> numpy[version='>=1.11.3,<2.0a0,>=1.14,<2.0a0'] -> python[version='>=3.6,<3.7.0a0,>=3.7,<3.8.0a0']
  - cuml -> libcuml[version='0.7.0.dev0.*,0.8.0a.*,0.9.0a.*'] -> cudf[version='0.7.*,0.8.*,0.9.*'] -> pyarrow=0.12.1 -> parquet-cpp=1.5.1 -> arrow-cpp[version='>=0.11.0,>=0.11.1,<0.12.0a0,>=0.12.0,>=0.12.0,<0.13.0a0'] -> numpy[version='>=1.11.3,<2.0a0,>=1.14,<2.0a0'] -> python[version='>=3.6,<3.7.0a0,>=3.7,<3.8.0a0']
  - dask-cudf -> di